In [1]:
import glob
import json

def getAllFilename(folder_name):
    filenames = []
    for file in glob.glob(folder_name):
        filenames.append(file)
    return filenames

In [2]:
def findIncompleteInfoDomain(filenames):
    incompleteDomain = {}
    for filename in filenames:
        with open(filename) as f_in:
            data = json.load(f_in)
            toolCount = data["data"]["enrichment"]["companyTools"]["count"]
            if toolCount > 30:
                domain_name = data["data"]["enrichment"]["domain"]
                incompleteDomain[domain_name] = toolCount
    return incompleteDomain

In [3]:
def writeAFile(filename, text):
    with open("stackshare_revise/"+filename, 'w') as f:
        f.write(text)

In [4]:
import requests

# request to Stackshare API
def postRequesttoStackShare(incompleteDomain):
  for key in incompleteDomain:
    print(key)
    stmt = '''
    {
      enrichment(domain: "'''+ key +'''"){
        domain
        companyId
        companyName
        companyTools (first:'''+ str(incompleteDomain[key]) +''', after:"") {
          count
          pageInfo {
            hasNextPage
            endCursor
          }
          edges {
            node {
              tool{
                id
                name
                category
                subCategory
              }
              sourcesSummary
              sources
            }
          }
        }
      }
    }
    '''
    
    url = "https://api.stackshare.io/graphql"
    headers = {
      "Content-Type": "application/json; charset=utf-8",
      "x-api-key": "UzwCp1M6UscAuv4lpFN9kA2345425345" #UzwCp1M6UscAuv4lpFN9kA
    }
    x = requests.post(url, headers=headers, json = {"query": stmt})
    if len(x.text)>2:
      writeAFile(key+".json", x.text)

In [ ]:
# inspect incomplete information
filenames = getAllFilename()
incompleteDomain = findIncompleteInfoDomain(filenames)
for key in incompleteDomain:
    print(key)
print("Total : ", len(incompleteDomain))

In [5]:
# add .json to all files
import os

def addJSONExt(filenames):
    for filename in filenames:
        os.rename(filename, filename+".json")

In [6]:
def readAllFiles(filenames):
    data = []
    for filename in filenames:
        print(filename)
        with open(filename, encoding="utf8") as f_in:
            data.append(json.load(f_in))
    return data
            # print(data["data"]["enrichment"]["domain"])
            # print(data["data"]["enrichment"]["companyName"])
            # print(data["data"]["enrichment"]["companyTools"]["edges"])

In [7]:
import pandas as pd

def extractData(data):
    dataframe = pd.DataFrame()
    for row in data:
        domain_name = row["data"]["enrichment"]["domain"]
        company_name = row["data"]["enrichment"]["companyName"]
        tools = row["data"]["enrichment"]["companyTools"]["edges"]
        for tool in tools:
            new_row = {
                'domain': domain_name, 
                'company name': company_name, 
                'tool': tool["node"]["tool"]["name"], 
                'category': tool["node"]["tool"]["category"],
                'subCategory': tool["node"]["tool"]["subCategory"]
            }
            df_row = pd.DataFrame(new_row, index=[0])
            dataframe = pd.concat([dataframe, df_row])
    return dataframe

In [14]:
# folder_name = "stackshare/*"
# folder_name = "stackshare/set2/*.json"
# folder_name = "stackshare/set2/apps/*.json"
# folder_name = "stackshare/set3/*.json"
folder_name = "stackshare/set5/*.json"
filenames = getAllFilename(folder_name)
data = readAllFiles(filenames)
dataframe = extractData(data)

stackshare/set5\roku.com.json


In [15]:
# export to csv
dataframe.to_csv('stackshare_info_5.csv', index=False)

In [ ]:
""